In [1]:
# script to get TTS for all UT segments 
# date created: 3/2/2020
# author: sofia chelpon

In [2]:
############################ PREP WORKSPACE ############################

In [4]:
import numpy as np 
import pandas as pd

import matplotlib
from matplotlib import pyplot as plt

%matplotlib inline
%config inlinebackend.figure_format='retina'

# import my TTS module, need to add path to folder
import sys
sys.path.insert(1, '/Volumes/scdrive2/TTS_2020/base_tts_code/')
import tts_mod

In [4]:
# read in ratios dataframe
path = "/mnt/home/sofia/TTS/paper_2020/get_tts/tts_vary_ut/data_prep/contrast_ratios_ut_seg.pkl"
contrast_ratios = pd.read_pickle(path)
info = contrast_ratios.iloc[[0, 1]]
segment_info = info.drop(columns = 'Instrument').drop(columns = 'BL_tau').drop(columns = 'TROPO_tau').drop(columns = 'UT_tau')
segment_info

,0,1,2,3,4,5,6,7,8,9,...,164,165,166,167,168,169,170,171,172,173
Time,2014-01-22 01:43:55,2014-01-22 01:55:10,2014-01-22 02:06:25,2014-01-22 02:17:40,2014-01-22 02:28:55,2014-01-22 02:37:51,2014-01-22 02:42:39,2014-01-22 02:53:54,2014-01-22 03:33:47,2014-01-22 03:48:46,...,2014-02-19 22:26:42,2014-02-22 00:36:54,2014-02-22 00:46:55,2014-02-22 00:56:54,2014-02-22 01:06:55,2014-02-22 01:15:26,2014-02-22 01:24:27,2014-02-22 01:33:26,2014-02-22 01:42:27,2014-02-22 01:51:27
Flight,RF05,RF05,RF05,RF05,RF05,RF05,RF05,RF05,RF05,RF05,...,RF13,RF14,RF14,RF14,RF14,RF14,RF14,RF14,RF14,RF14


In [5]:
# sort 
ratios_sort = contrast_ratios
ratios_sort = ratios_sort.drop('Time').drop('Flight')
ratios_sort = ratios_sort.sort_values(['BL_tau'])

# save ordered bl tau as a variable 
tau = ratios_sort['BL_tau']

# drop uncecessary columns 
ratios_sort = ratios_sort.drop(columns = ['Instrument', 'BL_tau', 'TROPO_tau', 'UT_tau'])

In [6]:
############################ GET TTS OUTPUTS FOR EA. CASE ############################
# --------------- test with a few seg 
# save outputs to plot later 
r2_allseg = []
mean_allseg = []
mode_allseg = []
num = []

utbl_allseg = pd.DataFrame()
mustar_allseg = pd.DataFrame()
gf_allseg = pd.DataFrame()
t_allseg = pd.DataFrame()
tau_allseg = pd.DataFrame()

for col in ratios_sort.columns: 
    utbl_allseg[col] = ''
    mustar_allseg[col] = ''
    gf_allseg[col] = ''
    t_allseg[col] = ''
    tau_allseg[col] = ''

In [7]:
############ --------------- LOOP THRU EACH SEGMENT, GET TTS OUTPUT --------------- ###########
# max length of G(t), usually 275999 but make higher so it doesnt fail 
topg = 276000

for col in ratios_sort:
    print(col)
    # ----- remove nans or else tts function fails 
    utbl_full = np.array(ratios_sort[col].values, dtype=np.float64)
    utbl_not_null_idx = np.argwhere(~np.isnan(utbl_full))
    # ----- inputs without nans 
    my_utbl = utbl_full[utbl_not_null_idx]
    my_tau = tau[utbl_not_null_idx]
    #
    # ----- run tts function 
    t, exp_decay_matrix, LT = tts_mod.prep_for_tts(my_tau)
    my_mustar, my_r2, my_gf, my_t, mean_age, mode_age = tts_mod.get_tts(my_utbl, my_tau, t, exp_decay_matrix, LT)
    #
    # ----- fill to make mu*, tau all length 52
    diffa = len(utbl_full) - len(my_utbl)
    if (diffa > 0):
        filla = np.empty((1,diffa))
        filla.fill(np.nan)
        my_tau = np.append(my_tau, filla)
        my_mustar = np.append(my_mustar, filla)
        my_utbl = np.append(my_utbl, filla)
    # ----- fill to make gf, t all length 275999
    diffb = 275999 - len(my_gf)
    if (diffb > 0):
        fillb = np.empty((1,diffb))
        fillb.fill(np.nan)
        my_gf = np.append(my_gf, fillb)
        my_t = np.append(my_t, fillb)
    #
    # ----- save individual values 
    r2_allseg.append(my_r2)
    mean_allseg.append(mean_age)
    mode_allseg.append(mode_age)
    # ----- save to dataframes
    utbl_allseg[col] = np.ndarray.flatten(my_utbl)
    mustar_allseg[col] = np.ndarray.flatten(my_mustar)
    tau_allseg[col] = np.ndarray.flatten(my_tau)
    gf_allseg[col] = np.ndarray.flatten(my_gf)
    t_allseg[col] = np.ndarray.flatten(my_t)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173


In [8]:
############ --------------- SAVE OUTPUTS TO PICKLE --------------- ###########
# make dataframe for r2, mean, mode, segment info 
r2_allseg = pd.Series(r2_allseg)
mean_allseg = pd.Series(mean_allseg)
mode_allseg = pd.Series(mode_allseg)

segment_info = segment_info.transpose()

segment_info['r squared'] = r2_allseg
segment_info['mean age'] = mean_allseg
segment_info['mode age'] = mode_allseg

segment_info.to_pickle('./segment_info.pkl')

# save others 
utbl_allseg.to_pickle('./utbl_allseg.pkl')
mustar_allseg.to_pickle('./mustar_allseg.pkl')
tau_allseg.to_pickle('./tau_allseg.pkl')
gf_allseg.to_pickle('./gf_allseg.pkl')
t_allseg.to_pickle('./t_allseg.pkl')